# Topic Modelling
## Example of the ’20 Newsgroups’ dataset and use LDA to extract discussed topics

In [1]:
import sys
import re, numpy as np, pandas as pd

In [2]:
# Gensim
import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import lemmatize, simple_preprocess
#from gensim.models import CoherenceModel
import matplotlib.pyplot as plt

In [3]:
%matplotlib inline
warnings.filterwarnings("ignore",category=DeprecationWarning)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

### We will use a portion of the 20 Newsgroups 
#### Let us select these topics from dataset: Christianity, Hockey, MidEast and Motorcycles.

In [15]:
# Import Dataset
df = pd.read_json('newsgroups.json')
df = df.loc[df.target_names.isin(['soc.religion.christian', 
                                  'rec.sport.hockey', 
                                  'talk.politics.mideast', 
                                  'rec.motorcycles']) , :]
print(df.shape)

(2361, 3)


In [16]:
pd.set_option('max_colwidth', 125)
df.head()

,content,target,target_names
10,From: irwin@cmptrc.lonestar.org (Irwin Arnstein)\nSubject: Re: Recommendation on Duc\nSummary: What's it worth?\nDistribu...,8,rec.motorcycles
10007,From: jet@netcom.Netcom.COM (J. Eric Townsend)\nSubject: Re: Insurance and lotsa points...\nIn-Reply-To: cjackson@adobe.c...,8,rec.motorcycles
10008,From: gld@cunixb.cc.columbia.edu (Gary L Dare)\nSubject: Re: ABC coverage\nNntp-Posting-Host: cunixb.cc.columbia.edu\nRep...,10,rec.sport.hockey
10017,From: nstramer@supergas.dazixco.ingr.com (Naftaly Stramer)\nSubject: Peace talks ...\nNntp-Posting-Host: supergas\nReply-...,17,talk.politics.mideast
10019,"From: mussack@austin.ibm.com (Christopher Mussack)\nSubject: Re: Questioning Authority\nLines: 60\n\nDespite my trendy, l...",15,soc.religion.christian


In [17]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 
                   'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 
                   'try', 'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 
                   'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 
                   'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

### Tokenize and Clean
#### Removing the emails, new line characters, single quotes and finally split the sentence into a list of words

In [18]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub('\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub('\s+', ' ', sent)  # remove newline chars
        sent = re.sub("\'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True) 
        yield(sent)

In [19]:
# Convert to list
data = df.content.values.tolist()
data_words = list(sent_to_words(data))
print(data_words[:1])

[['from', 'irwin', 'arnstein', 'subject', 're', 'recommendation', 'on', 'duc', 'summary', 'whats', 'it', 'worth', 'distribution', 'usa', 'expires', 'sat', 'may', 'gmt', 'organization', 'computrac', 'inc', 'richardson', 'tx', 'keywords', 'ducati', 'gts', 'how', 'much', 'lines', 'have', 'line', 'on', 'ducati', 'gts', 'model', 'with', 'on', 'the', 'clock', 'runs', 'very', 'well', 'paint', 'is', 'the', 'bronze', 'brown', 'orange', 'faded', 'out', 'leaks', 'bit', 'of', 'oil', 'and', 'pops', 'out', 'of', 'st', 'with', 'hard', 'accel', 'the', 'shop', 'will', 'fix', 'trans', 'and', 'oil', 'leak', 'they', 'sold', 'the', 'bike', 'to', 'the', 'and', 'only', 'owner', 'they', 'want', 'and', 'am', 'thinking', 'more', 'like', 'any', 'opinions', 'out', 'there', 'please', 'email', 'me', 'thanks', 'it', 'would', 'be', 'nice', 'stable', 'mate', 'to', 'the', 'beemer', 'then', 'ill', 'get', 'jap', 'bike', 'and', 'call', 'myself', 'axis', 'motors', 'tuba', 'irwin', 'honk', 'therefore', 'am', 'computrac', 'r

### Build the Bigram, Trigram Models and Lemmatize

In [24]:
# Build the bigram and trigram models
# higher threshold fewer phrases.
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) 
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

### Lemmatize each word to its root form, keeping only nouns, adjectives, verbs and adverbs

In [25]:
def process_words(texts, stop_words=stop_words, 
                  allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) 
              if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en', disable=['parser', 'ner'])
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) 
                  if word not in stop_words] for doc in texts_out]    
    return texts_out

data_ready = process_words(data_words)

### Build the Topic Model

In [26]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)
print(id2word)

Dictionary(24628 unique tokens: ['accel', 'arnstein', 'axis', 'beemer', 'bike']...)


In [27]:
# Create Corpus: Term Document Frequency
corpus = [id2word.doc2bow(text) for text in data_ready]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 2), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 2), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1)]]


#### In the above corpus (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 2 occurs twice and so on. 
#### If you want to see what word a given id corresponds to, pass the id as a key to the dictionary.

In [28]:
id2word[0]

'accel'

#### chunksize is the number of documents to be used in each training chunk. 
#### update_every determines how often the model parameters should be updated 
#### passes is the total number of training passes.

In [29]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=4, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10,
                                           passes=10,
                                           alpha='symmetric',
                                           iterations=100,
                                           per_word_topics=True)

In [30]:
print(lda_model.print_topics())

[(0, '0.021*"team" + 0.021*"game" + 0.015*"hockey" + 0.012*"nhl" + 0.011*"player" + 0.010*"play" + 0.010*"win" + 0.010*"hawk" + 0.010*"season" + 0.008*"year"'), (1, '0.020*"armenian" + 0.011*"israel" + 0.009*"israeli" + 0.008*"people" + 0.008*"state" + 0.007*"kill" + 0.006*"article" + 0.006*"government" + 0.005*"turk" + 0.005*"center"'), (2, '0.010*"people" + 0.009*"god" + 0.007*"christian" + 0.007*"believe" + 0.007*"write" + 0.007*"time" + 0.006*"question" + 0.006*"law" + 0.005*"organization" + 0.005*"book"'), (3, '0.012*"organization" + 0.012*"write" + 0.011*"article" + 0.009*"time" + 0.009*"bike" + 0.008*"new" + 0.007*"well" + 0.007*"look" + 0.004*"university" + 0.004*"bad"')]


### What is the Dominant topic and its percentage contribution in each document

In [31]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(
                    pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), 
                    ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [32]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, 
                                                  texts=data_ready)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 
                             'Keywords', 'Text']
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,3.0,0.8938,"organization, write, article, time, bike, new, well, look, university, bad","[irwin, arnstein, recommendation, duc, summary, worth, expire, sit, gmt, organization, computrac, inc, richardson_tx, key..."
1,1,3.0,0.6377,"organization, write, article, time, bike, new, well, look, university, bad","[eric, townsend, insurance, lotsa_point, reply, message, mon, organization, netcom_online, communication, service, cjacks..."
2,2,0.0,0.4612,"team, game, hockey, nhl, player, play, win, hawk, season, year","[gary_dare, reply, gary_dare, organization, phds, hall, caleb, cohen, write, boy, everyone, rip, espns, hockey, coverage,..."
3,3,1.0,0.6750,"armenian, israel, israeli, people, state, kill, article, government, turk, center","[naftaly_stramer, peace_talk, reply, organization, israeline, today, maariv, report, yesterday, follow, egyptian, preside..."
4,4,2.0,0.7443,"people, god, christian, believe, write, time, question, law, organization, book","[question, authority, trendy, liberal, feminist, tendency, fact, basically, agree, rebut, write, chris_mussack, write, pe..."
5,5,2.0,0.6329,"people, god, christian, believe, write, time, question, law, organization, book","[harrasse, work, prayer, reply, organization, university, chicago, repeat, emphasize, someone_else, feel, horrible, worth..."
6,6,3.0,0.8103,"organization, write, article, time, bike, new, well, look, university, bad","[andrew_infante, little_hasty, organization, duke, university, durham, apparently, last, little, hasy, call, place, quote..."
7,7,2.0,0.9511,"people, god, christian, believe, write, time, question, law, organization, book","[james_sledd, afterlife, organization, social, science, computing, way, look, die, release, arc, time, able, comprehend, ..."
8,8,0.0,0.5199,"team, game, hockey, nhl, player, play, win, hawk, season, year","[canadien, stanley_cup, organization, canada, dean, pereira, write, kind, team, montreal, cup, problem, everyone, steal, ..."
9,9,0.0,0.8704,"team, game, hockey, nhl, player, play, win, hawk, season, year","[steve_gallichio, possible, canadian, wc, team, organization, cadkey_inc, point, team, canadian, nhler, playoff, bind, ce..."


### Visualize

In [33]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary=lda_model.id2word)
vis

/Users/gurukrupa/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.176177 -0.101051       1        1  36.498779
3      0.058033 -0.167053       2        1  26.701822
1      0.111872  0.262655       3        1  23.408344
0     -0.346082  0.005450       4        1  13.391055, topic_info=      Category         Freq        Term        Total  loglift  logprob
term                                                                  
1479   Default  1493.000000    armenian  1493.000000  30.0000  30.0000
521    Default   890.000000        team   890.000000  29.0000  29.0000
1180   Default   889.000000        game   889.000000  28.0000  28.0000
155    Default   609.000000      hockey   609.000000  27.0000  27.0000
706    Default   798.000000      israel   798.000000  26.0000  26.0000
1408   Default   502.000000         nhl   502.000000  25.0000  25.0000
391    Default   978.000000         god   978.000000  24.0000  24.0000
4      Default   738.000000        bike   738.000000  23.0000  23.0000
2283   Default   666.000000     israeli   666.000000  22.0000  22.0000
2488   Default   461.000000      player   461.000000  21.0000  21.0000
230    Default  1777.000000      people  1777.000000  20.0000  20.0000
97     Default   436.000000        play   436.000000  19.0000  19.0000
524    Default   433.000000         win   433.000000  18.0000  18.0000
2907   Default   423.000000        hawk   423.000000  17.0000  17.0000
544    Default   404.000000      season   404.000000  16.0000  16.0000
965    Default   810.000000   christian   810.000000  15.0000  15.0000
379    Default   795.000000     believe   795.000000  14.0000  14.0000
87     Default   505.000000        kill   505.000000  13.0000  13.0000
242    Default   758.000000       state   758.000000  12.0000  12.0000
467    Default   739.000000        look   739.000000  11.0000  11.0000
718    Default   656.000000         law   656.000000  10.0000  10.0000
324    Default   871.000000         new   871.000000   9.0000   9.0000
1064   Default   611.000000        book   611.000000   8.0000   8.0000
676    Default   421.000000  government   421.000000   7.0000   7.0000
49     Default  1031.000000        well  1031.000000   6.0000   6.0000
188    Default  1593.000000        time  1593.000000   5.0000   5.0000
1644   Default   391.000000        turk   391.000000   4.0000   4.0000
2479   Default   273.000000      league   273.000000   3.0000   3.0000
1542   Default   373.000000    genocide   373.000000   2.0000   2.0000
417    Default   540.000000        talk   540.000000   1.0000   1.0000
...        ...          ...         ...          ...      ...      ...
10096   Topic4   160.504654          sj   161.206924   2.0062  -5.5622
4917    Topic4   160.204208         det   160.906097   2.0062  -5.5641
4962    Topic4   159.467041         pit   160.170059   2.0062  -5.5687
4985    Topic4   156.657974         tor   157.360321   2.0061  -5.5865
3896    Topic4   156.194489         stl   156.896301   2.0061  -5.5895
1275    Topic4   161.417084  pittsburgh   162.148926   2.0061  -5.5566
4905    Topic4   152.336945         buf   153.038147   2.0060  -5.6145
1376    Topic4   162.491333  defenseman   163.283127   2.0057  -5.5499
1903    Topic4   136.437225         chi   137.139465   2.0054  -5.7247
3743    Topic4   137.358170     detroit   138.065292   2.0054  -5.7180
8462    Topic4   138.087128         har   138.799942   2.0054  -5.7127
4303    Topic4   150.245834         ahl   151.023422   2.0054  -5.6283
4470    Topic4   188.805298      norris   189.782547   2.0054  -5.3998
1394    Topic4   137.686920         ice   138.413132   2.0053  -5.7156
8482    Topic4   132.360413         phi   133.064224   2.0053  -5.7550
4964    Topic4   130.750778         que   131.453308   2.0052  -5.7673
14683   Topic4   126.201591          tb   126.903740   2.0050  -5.8027
2907    Topic4   420.062988        hawk   423.289429   2.0029  -4.6002
93      Topi